In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 80.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
from haystack.utils import launch_es
launch_es()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=728,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/dev_rev_data/train_data.csv')
df['Id']=[i for i in range(df.shape[0])]
df.head()

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,Id
0,2,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4,...",When did Beyonce leave Destiny's Child and become a solo singer?,True,['2003'],[526],0
1,6,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4,...",What album made her a worldwide known artist?,True,['Dangerously in Love'],[505],1
2,7,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4,...",Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360],2
3,8,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4,...",When did Beyoncé rise to fame?,True,['late 1990s'],[276],3
4,9,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4,...",What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290],4


In [ ]:
data_path='/content/data'
try:
  os.mkdir(data_path)
except:
  print("Here")

In [ ]:
import os
def df_to_txt(df,path='/content/data',split_theme=False):
  if split_theme==True:
    for i in range(df.shape[0]):
      if i %1000 == 0:
        print("At i = ",i)
      rw=df.iloc[i];
      data_path=path+'/'+rw['Theme']
      try:
        os.mkdir(data_path)
      except:
        pass
      file_path=data_path+'/'+str(rw['Id'])+'.txt'
      file1 = open(file_path,"w+")
      file1.write(rw['Paragraph'])
      file1.close()
  else:
    for i in range(df.shape[0]):
      if i %1000 == 0:
        print("At i = ",i)
      rw=df.iloc[i];
      data_path=path+'/full'
      try:
        os.mkdir(data_path)
      except:
        pass
      file_path=data_path+'/'+str(rw['Id'])+'.txt'
      file1 = open(file_path,"w+")
      file1.write(rw['Paragraph'])
      file1.close()
      




In [ ]:
theme_list=df['Theme'].unique().tolist()
len(theme_list)
dict_doc_store={}
index_pipeleine_store={}
dict_retiever={}
for theme in theme_list:
  dict_doc_store[theme]=0
  index_pipeleine_store[theme]=0
  dict_retiever[theme]=0
dict_id_to_context={}

In [ ]:
def train_retriver(df,path='/content/data',by_theme=False):
  df_to_txt(df,path,by_theme)
  doc_dir='/content/data/'
  if by_theme==True:
    for theme in theme_list:
      print("At theme = ",theme)
      document_store_temp = ElasticsearchDocumentStore(
        host=host,
        username="",
        password="",
        index="document"
      )
      doc_dir_tem=doc_dir+theme
      indexing_pipeline_temp = Pipeline()
      text_converter = TextConverter()
      preprocessor = PreProcessor(
          clean_whitespace=True,
          clean_header_footer=True,
          clean_empty_lines=True,
          split_by="word",
          split_length=728,
          split_overlap=20,
          split_respect_sentence_boundary=True,
      )
      indexing_pipeline_temp.add_node(component=text_converter, name="TextConverter", inputs=["File"])
      indexing_pipeline_temp.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
      indexing_pipeline_temp.add_node(component=document_store_temp, name="DocumentStore", inputs=["PreProcessor"])
      files_to_index = [doc_dir_tem + "/" + f for f in os.listdir(doc_dir_tem)]
      resk=indexing_pipeline_temp.run_batch(file_paths=files_to_index)
      for i in resk['documents']:
        dict_id_to_context[i.id]=i.content
      dict_doc_store[theme]=document_store_temp
      index_pipeleine_store[theme]=indexing_pipeline_temp
  else:
    document_store_full = ElasticsearchDocumentStore(
      host=host,
      username="",
      password="",
      index="document"
    )
    doc_dir+='full'
    indexing_pipeline_full = Pipeline()
    text_converter = TextConverter()
    preprocessor = PreProcessor(
        clean_whitespace=True,
        clean_header_footer=True,
        clean_empty_lines=True,
        split_by="word",
        split_length=728,
        split_overlap=20,
        split_respect_sentence_boundary=True,
    )
    indexing_pipeline_full.add_node(component=text_converter, name="TextConverter", inputs=["File"])
    indexing_pipeline_full.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
    indexing_pipeline_full.add_node(component=document_store_full, name="DocumentStore", inputs=["PreProcessor"])
    files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
    resk=indexing_pipeline_full.run_batch(file_paths=files_to_index)
    for i in resk['documents']:
      dict_id_to_context[i.id]=i.content
    dict_doc_store['full']=document_store_full
    index_pipeleine_store['full']=indexing_pipeline_full





    

In [ ]:
train_retriver(df,by_theme=True)

In [ ]:
files_to_index = ['/content/data/full' + "/" + f for f in os.listdir('/content/data/full')]
resk=indexing_pipeline.run_batch(file_paths=files_to_index[0:4])

In [ ]:
for i in resk['documents']:
  print(i.id)
  print(i.content)

NameError: ignored

In [ ]:
dict_doc_store['full']

In [ ]:
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="deepset/tinyroberta-squad2", use_gpu=True)
from haystack import Pipeline


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/tinyroberta-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/tinyroberta-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
if 'full' in theme_list:
  print('present')
else :
  theme_list.append('full')

In [ ]:
dict_query_pipeline={}
for theme in theme_list:
  dict_retiever[theme]=BM25Retriever(document_store=dict_doc_store[theme])
  querying_pipeline = Pipeline()
  querying_pipeline.add_node(component=dict_retiever[theme], name="Retriever", inputs=["Query"])
  querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])
  dict_query_pipeline[theme]=querying_pipeline


In [ ]:
def predict(theme,question,top_k_reader=1,top_k_retriever=2):
  start=time.time()
  querying_pipeline=dict_query_pipeline[theme]
  # querying_pipeline = Pipeline()
  # querying_pipeline.add_node(component=retiever, name="Retriever", inputs=["Query"])
  # querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])
  prediction = querying_pipeline.run(
      query=question,
      params={
          "Retriever": {"top_k": top_k_retriever},
          "Reader": {"top_k": top_k_reader}
      }
  )
  best_pred=[0,"none","doc","id"]
  for ans in prediction['answers']:
    if best_pred[0]<=ans.score:
      best_pred=[ans.score,ans.answer,dict_id_to_context[ans.document_id],ans.document_id]

  print("Els time = ",time.time()-start)

  return [best_pred,time.time()-start]



In [ ]:
retirever=dict_retiever['full']
ls=[reader,retirever]
preds=predict('full',"How long have international oil companies been in Nigeria?",2,2)


In [ ]:
preds

[[0.9121561050415039,
  'decades',
  'The Niger Delta Nembe Creek Oil field was discovered in 1973 and produces from middle Miocene deltaic sandstone-shale in an anticline structural trap at a depth of 2–4 km. In June 2013, the company announced a strategic review of its operations in Nigeria, hinting that assets could be divested. While many international oil companies have operated there for decades, by 2014 most were making moves to divest their interests, citing a range of issues including oil theft. In August 2014, Shell Oil Company said it was finalising its interests in four Nigerian oil fields.',
  'f72829c4f17176a9abac4de8abac8279'],
 2.0400357246398926]

Dont Use Code after This cell

In [ ]:
%pip install -U gensim
%pip install ray
%pip install torch
%pip install transformers
%pip install scikit-learn
%pip install psutil

In [ ]:
theme='Nigeria'
df_new=df[df["Theme"]==theme]
df_new.head()

In [ ]:
questions=df_new['Question'].tolist()
for i in range(len(questions)):
  questions[i]=[theme,questions[i]]
len(questions)

180

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from transformers import pipeline
import time
from datetime import timedelta
import psutil
import ray
num_cpus = psutil.cpu_count(logical=True)
print('Number of available CPUs:', num_cpus)

Number of available CPUs: 2


In [ ]:
num_cpus = psutil.cpu_count(logical=True)
print('Number of available CPUs:', num_cpus)
st=time.time()
# Start Ray cluster
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)
pipe_id = ray.put(reader)

# @ray.remote decorator enables to use this 
# function in distributed setting
@ray.remote
def predict_pipe(full_pipeline,question,pipeline):
    return predict(full_pipeline,theme,question,2,2)

en = time.time()
print('Loading Time :', str(timedelta(seconds=en-st)))


2023-01-26 17:27:50,699	INFO worker.py:1370 -- Calling ray.init() again after it has already been called.


Number of available CPUs: 2


TypeError: ignored

In [ ]:
def multi_process_predict(inputs):
    start = time.time()
    # Run the function using multiple cores and gather the results
    predictions = ray.get([predict_pipe.remote(pipeline,theme,question) for theme,question,pipeline in inputs])
    end = time.time()
    print('Prediction time:', str(timedelta(seconds=end-start)))
    
    return predictions,end-start

In [ ]:
final_pred=multi_process_predict(questions)